### jupyter Settings

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
%load_ext autoreload
%autoreload 2

#### Read Dataset

In [ ]:
import pandas as pd

data_files = ['../data/youtube-spam/Youtube01-Psy.csv','../data/youtube-spam/Youtube02-KatyPerry.csv','../data/youtube-spam/Youtube03-LMFAO.csv','../data/youtube-spam/Youtube04-Eminem.csv','../data/youtube-spam/Youtube05-Shakira.csv']
train_data = []
for file in data_files:
    data = pd.read_csv(file)
    train_data.append(data)
train_data = pd.concat(train_data)
def drop_fectures(features,data):
    data.drop(features,axis=1,inplace=True)
drop_fectures(['COMMENT_ID','AUTHOR','DATE'],train_data)
train_data['processed_content'] = train_data['CONTENT']

drop_fectures(['CONTENT'],train_data)

In [ ]:
train_data[train_data.CLASS == 1].shape

In [ ]:
train_data[train_data.CLASS == 0].shape

In [ ]:
train_data.shape

In [ ]:
train_data

In [ ]:
df = train_data
df.loc[df.CLASS == 1, 'CLASS'] = 'malicious'
df.loc[df.CLASS == 0, 'CLASS'] = 'normal'
df.rename(columns={"CLASS": "target",'processed_content':'text' },inplace=True)

In [ ]:
df

In [ ]:
df.index

#### Train model

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from tpe_model import text_preprocess
from tpe_model import text_model_generator
df, label_map = text_preprocess(df)
print(label_map)
tmg = text_model_generator(df)
model = tmg.model_trainer()

#### Explain single sample

In [ ]:
from tpe_core import get_instance_explained
from lime.lime_text import LimeTextExplainer

labels = list(label_map.values())
get_instance_explained(df, 1, model, label_map, 'malicious')

In [ ]:
get_instance_explained(df, 0, model, label_map, 'normal')

#### Generate signature rules in batch and verification

In [ ]:
from tpe_rule_validation import rule_matching_evaluation
match_result, rules_tobe_validate, matched_rules = rule_matching_evaluation(df
                         , seed_num=1000
                         , rein_num=500
                         , eval_num=1500
                         , model=model
                         , label_map=label_map
                         , refer_label='malicious'
                         , lime_flag = True
                         , scan_flag=True
                         , content_direction='backward'
                         , xcol_name='text'
                         , n_cores=10)

In [ ]:
# show a case
print('A match case...')
rule_index = 1
rule_num = matched_rules.iloc[[rule_index]].index[0]
rule_num = 49
print('rule_num is %d' % rule_num)
print(matched_rules.loc[[rule_num]]['rule_strings'])
print('----------------------------------------------------------------------')

pd.options.display.max_colwidth = 1000
print(match_result.loc[match_result.rule_num == rule_num]['text'])
print('Total matched number %d' % match_result.loc[match_result.rule_num == rule_num].shape[0])

In [ ]:
matched_rules.shape[0]